## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

In [1]:
import pandas as pd
import numpy as np
import keras
import zipfile as zf
%matplotlib inline
import os

import matplotlib.pyplot as plt
import pandas as pd
import cv2
import numpy as np
from glob import glob
import seaborn as sns

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalMaxPooling2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

Using TensorFlow backend.


### Load Dataset Files

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [14]:
%cd /gdrive/'My Drive/Data/CNN2'

/gdrive/My Drive/Data/CNN2


Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [9]:
project_path

"/gdrive/'My Drive/Data/CNN2"

In [0]:
project_path = "/gdrive/'My Drive/Data/CNN2"

In [0]:
file = zf.ZipFile('train.zip')
file.extractall()


Repeat the same step for test.zip

In [0]:
file1 = zf.ZipFile('test.zip')
file1.extractall()

Repeat the same step for sample_submission.csv.zip

In [0]:
file = zf.ZipFile('sample_submission.csv.zip')
file.extractall()

Repeat the same step for labels.csv.zip

In [0]:
file = zf.ZipFile('labels.csv.zip')
file.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [0]:
labels = pd.read_csv("labels.csv")

### Print the count of each category of Dogs given in the dataset



In [0]:
breed_counts = labels['breed'].value_counts()

In [33]:
breed_counts.head()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
Name: breed, dtype: int64

In [34]:
breed_counts.shape

(120,)

### Get one-hot encodings of labels

In [0]:
targets = pd.Series(labels['breed'])
one_hot = pd.get_dummies(targets,sparse=True)
one_hot_label = np.asarray(one_hot)

In [40]:
one_hot_label[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [0]:
img_rows =128
img_columns= 128
num_channels =1

In [44]:
import cv2
from tqdm import tqdm
x_train = []
y_train =[]

for f,img in tqdm(labels.values):
  train_img =cv2.imread('./train/{}.jpg'.format(f),1)
  train_img_resize =cv2.resize(train_img,(img_rows,img_columns))
  x_train.append(train_img_resize)
  y_train.append(img)

100%|██████████| 10222/10222 [01:11<00:00, 142.51it/s]


Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [0]:
def resize_images(img):
  img = np.array(img).astype(np.uint8)
  res = cv2.resize(img,(256,256), interpolation = cv2.INTER_CUBIC)
  return res

In [0]:
x_train = [resize_images(img) for img in x_train]


In [0]:
from sklearn.preprocessing import LabelBinarizer
y = LabelBinarizer().fit_transform(y_train)


### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [0]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_train, y, test_size=0.3, random_state=7)

In [0]:
X_test, X_val, y_test, Y_val = train_test_split(X_test,y_test, test_size = 0.5, random_state=7)

### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [0]:
test_img = pd.read_csv('sample_submission.csv').id

In [62]:
test_img.head()

0    000621fb3cbb32d8935728e48679680e
1    00102ee9d8eb90812350685311fe5890
2    0012a730dfa437f5f3613fb75efcd4ce
3    001510bc8570bbeee98c8d80c8a95ec1
4    001a5f3114548acdefa3d4da05474c2e
Name: id, dtype: object

Run the below code to load the test image files in x_test_feature

In [64]:
x_test_feature = []
i = 0 # initialisation
for f in tqdm(test_img.values): # f for format ,jpg
    img = cv2.imread('./test/{}.jpg'.format(f), 0)
    img_resize = cv2.resize(img, (img_rows, img_columns)) 
    x_test_feature.append(img_resize)

100%|██████████| 10357/10357 [01:00<00:00, 171.91it/s]


Normalize the test data and convert it into 4 dimensions

In [0]:
x_test_feature = [resize_images(img) for img in x_test_feature]

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [0]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5), activation ='relu', batch_input_shape = (5,256, 256, 3)))

model.add(Conv2D(filters = 32, kernel_size = (3,3), activation ='relu'))

model.add(Dense(256, activation = "relu"))

model.add(Dense(120, activation = "softmax"))


In [104]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (5, 252, 252, 32)         2432      
_________________________________________________________________
conv2d_31 (Conv2D)           (5, 250, 250, 32)         9248      
_________________________________________________________________
dense_19 (Dense)             (5, 250, 250, 256)        8448      
_________________________________________________________________
dense_20 (Dense)             (5, 250, 250, 120)        30840     
Total params: 50,968
Trainable params: 50,968
Non-trainable params: 0
_________________________________________________________________


### Use batch_size = 128 and epochs = 10 and execute the model

In [0]:
model.compile(optimizer = 'Adam', loss = "categorical_crossentropy", metrics = ["accuracy"])

In [107]:
import numpy as np
model.fit(np.array(X_train), np.array(y_train), epochs = 10, validation_data = (np.array(X_val),np.array(Y_val)),batch_size = 120)

ValueError: ignored

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [0]:
from keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=(project_path+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg')

Print the summary of the base_model

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

### Make all the layers in the base_model (VGG16) to be non-trainable

### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%